`GB` BigData / [Олег Гладкий](https://gb.ru/users/3837199) // домашнее задание

`262698` __Методы сбора и обработки данных из сети Интернет__:  `04`. __MongoDB__ в Python

# Домашнее задание по теме: MongoDB в Python

1. Развернуть у себя _(на компьютере/виртуальной машине/хостинге)_ базу данных NoSQL __MongoDB__ и реализовать функцию, которая будет добавлять только _новые_ вакансии/продукты в вашу базу.

2. (*) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

3. (*) Любая аналитика. Например matching ваканский с разных площадок



In [1]:
import json
import pandas as pd
from pprint import pprint
import pymongo
import sys

## Задание 1

* Развернуть у себя на компьютере базу данных NoSQL __MongoDB__.
* Реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

### Данные для БД

__Читаем__ данных с диска, полученные в предыдущем дамашнем задании «Парсинг даных: HTML, Beautiful-Soap». С сайта `hh.com`. Данные в файле `03_hw_HTML_Beautiful-Soap_VACANCIES.json` в `json`-формате.

In [2]:
vacancies_dict = dict({})
vacancies = list([])

# Читаем результаты парсинга сайта hh.ru (читаем сразу всё, а не частично)

with open('03_hw_HTML_Beautiful-Soap_VACANCIES.json', 'r', encoding='utf-8') as f:
    vacancies_list = json.load(f)
    
# vacancies_list

### Создание БД

Начальная работа с базой данных `MongoDB`: инициализируем ссылки на объекты и т. д.

In [3]:
# Подключаемся к серверу баз данных

client = pymongo.MongoClient('mongodb://127.0.0.1:27017') 

# Удаляем базу данных, перед начало работы с ней НА ДАННОМ ЭТАПЕ -- новые данные парсинга отсутствуют
client.drop_database('hh')

# Задаём ссылку на базу данных
db = client.hh

# Создаём ссылку на коллекцию
vacancies = db.vacancies

# Проверяем имеющиеся коллекции (таблицы) в БД по нашей ссылке
db.list_collection_names()

[]

### Индекс

Создадим индекс по полю `Link`, включающий `id` вакансии, и исключим дублирование документов БД. В нашем случае этот индекс является дополнительным к уже имеющемуся основному индексу ObjectID самой базы данных.
Инфо: https://pymongo.readthedocs.io/en/stable/tutorial.html

In [4]:
# Для этой коллекции создаём индекс (у меня mongo 4.2.23)

index_link = vacancies.create_index([('Link', pymongo.ASCENDING)], unique=True)
index_link, type(index_link)

('Link_1', str)

### Вносим данные в БД

Вносим __только новые__ вакансии: дублирование обрабатываем и отбрасываем.

Данные, полученные ввиде json-файла из предыдущего урока, вносим в базу. 

In [5]:
i_err_dup = 0
i_err_other = 0
i_ok = 0
for i, vacancy in enumerate(vacancies_list, start=1):
    try:
        vacancies.insert_one(vacancy)                   # заносим документы в базу по одному
        
    except pymongo.errors.DuplicateKeyError:
        i_err_dup += 1
    except:
        i_err_other += 1
    else: 
        i_ok += 1

print(f"-----")  # Отчитываемся
print(f"Всего обработано {i} документов: "
      f"успешно {i_ok}, ",
      f"дублирование {i_err_dup}, ", 
      f"другие ошибки {i_err_other}.")
print(f"Запрос документов в коллекции БД, всего: {vacancies.count_documents({})}")

-----
Всего обработано 1160 документов: успешно 1145,  дублирование 15,  другие ошибки 0.
Запрос документов в коллекции БД, всего: 1145


## Задание 2

Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы, при этом необходимо следующее:
* необходимо анализировать оба поля зарплаты.

### Задайте уровень дохода!

Необходимо задать:
* уровень дохода `pay_level_ru` в рублях
* курс доллара `rate_us_dollar`

Полученная выбрка вакансий будет (в том числе) выведена в файл `04_hw_MongoDB_Python__pay_selection.txt` на диск.

In [6]:
pay_level_ru = 400000       # Задайте зарплату

rate_us_dollar = 60         # Курс доллара 


# Пересчёт уровня в рубли для "долларовых" вакансий

pay_level_us = pay_level_ru / rate_us_dollar
curr_ru='руб'
curr_us='USD'

#### Формирование запроса

Сформируем запрос для получения выборки (в виде курсора на эти данные). При этом учём соглашения, принятые для соответствия указаниям сайта и значениям словаря на примере:
* от 1000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 0
* от 1000 до 3000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 3000 
* до 3000 р.
    * `pay['Maney_min']` = 0
    * `pay['Maney_max']` = 3000
* 1000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 1000
    
Регулярные выражения: https://www.mongodb.com/docs/manual/reference/operator/query/regex/


In [7]:
vacancies_selected = vacancies.find(
    {'$or': [{'$and': [{'Maney_min': {'$eq': 0}},               # Рубли
                       {'Maney_max': {'$gt': pay_level_ru}},
                       {'Maney_curr': {'$regex': curr_ru, '$options': 'i'}}
                      ]
             },              
             
             {'$and': [{'Maney_min': {'$gt': pay_level_ru}}, 
                       {'Maney_max': {'$eq': 0}},
                       {'Maney_curr': {'$regex': curr_ru, '$options': 'i'}}
                      ]
             },
             
             {'$and': [{'Maney_min': {'$gt': 0}}, 
                       {'Maney_max': {'$gt': pay_level_ru}},
                       {'Maney_curr': {'$regex': curr_ru, '$options': 'i'}}
                      ]
             },
     
             {'$and': [{'Maney_min': {'$eq': 0}},               # USD
                       {'Maney_max': {'$gt': pay_level_us}},
                       {'Maney_curr': {'$regex': curr_us, '$options': 'i'}}
                      ]
             },              
             
             {'$and': [{'Maney_min': {'$gt': pay_level_us}}, 
                       {'Maney_max': {'$eq': 0}},
                       {'Maney_curr': {'$regex': curr_us, '$options': 'i'}}
                      ]
             },
             
             {'$and': [{'Maney_min': {'$gt': 0}}, 
                       {'Maney_max': {'$gt': pay_level_us}},
                       {'Maney_curr': {'$regex': curr_us, '$options': 'i'}}
                      ]
             },
             
            ]
    }
)
 

#### Выводим результат
Результат выводи на экран в стандартный поток вывода и в файл на диск...

In [8]:
with open('04_hw_MongoDB_Python__pay_selection.txt', 'a', encoding='utf-8') as f_out:
    strims = [sys.stdout, f_out]  # выводим в файл и стандартный поток вывода (для удобства)
    
    for strim in strims:
        print(f"PAY-LEVEL:{pay_level_ru}", file=strim)
        
    for i, vacancy in enumerate(vacancies_selected, start=1):
        for strim in strims:
            print(f"{i:3}  {vacancy['Maney_min']:6}-{vacancy['Maney_max']:6} ",
                  f"{vacancy['Maney_curr']:4}  {vacancy['Name']}", 
                  file=strim)

PAY-LEVEL:400000
  1    6000-  8000  USD   Разработчик C++ (релокация в Dubai)
  2  400000-450000  руб.  Python Team Lead / Руководитель backend разработки (в аккредитованную ИТ компанию)
  3    9500-  9500  USD   Manager, Site Reliability Engineering (to Canada/Serbia)
  4    7000-  7000  USD   Data Analyst - NLP (to Canada/Serbia)
  5    8000-  8000  USD   Site Reliability Engineer (to Canada/Serbia)
  6    8000-  8000  USD   Senior DevOps Engineer - Automation (to Canada/Serbia)
  7   10000- 10000  USD   Head of DevOps (Dubai)
  8  250000-450000  руб.  Fullstack техлид (Python)
  9    5500-  7200  USD   Product Owner in QA
 10  400000-700000  руб.  Senior Software Engineer
 11  600000-600000  руб.  Разработчик Solidity/Разработчик Rust/Разработчик Go/Разработчик python(удаленно)
 12  400000-700000  руб.  Программист С++ Senior
 13  300000-500000  руб.  Разработчик C# / Python
 14    5000-  9000  USD   Senior Backend Engineer (Remote, to Canada/USA)


<!--  -->

<!--  -->

<!--  -->

__P.S.__



Почему-то вроде-бы такая простая задача получается такой сложной в исполнении?

При контроле внесения дубля вакансии не отработана возможность обновления записи, так как дубль может быть более свежей записью, хотя по логике это не так (более свежие в начале файла).

<!--  -->

<!--  -->

<!--  -->

ZIP of code...

In [9]:
%%time
# инициализируем выходную таблицу
# vacancies = pd.DataFrame( \
#     columns=['Name', 'Company', 'Link', 'Date', 'Source', 'Maney_min', 'Maney_max', 'Maney_curr'], \
#     index=[])

Wall time: 0 ns
